<a href="https://colab.research.google.com/github/SophieShin/MachineLearning/blob/main/Option_Pricing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### 반드시 사본 저장 후 사본파일에서 실행해주세요.(파일/Drive에 사본저장)
#### 아래 '재생'버튼을 누르면서 진행하면 됩니다


In [196]:
# 필요한 라이브러리 import
import pandas as pd
import numpy as np

#### [CR20]화면에서 검증하고자 하는 상품의 이론가격 파라미터를 저장합니다
#### 결제월 : 전체
#### 선물옵션구분: 콜+풋
#### 이론가용도: 기준가용
#### 화면에서 프린트 버튼을 누른 뒤 엑셀로 저장하면 됩니다.
#### 엑셀로 저장할 때  'CSV' 형식으로 저장합니다.

In [263]:
# 파일 업로드
from google.colab import files

uploaded = files.upload()

Saving 이론가파람_SK하이닉스.csv to 이론가파람_SK하이닉스 (3).csv


#### 아래 '이론가파람_SK하이닉스' 부분을 자신이 저장한 파일 이름으로 변경합니다

In [264]:
import io
df = pd.read_csv(io.BytesIO(uploaded['이론가파람_SK하이닉스.csv']), thousands = ',')
df

,종목명,Unnamed: 1,선물옵션\n구분,Unnamed: 3,최종\n거래일,Unnamed: 5,행사가격,Unnamed: 7,기초자산\n기준가격,Unnamed: 9,...,배당 현재가치,Unnamed: 17,변동성,Unnamed: 19,Unnamed: 20,이론가격,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25
0,"C 202307 78,000( 10)",NaN,콜,NaN,20230713,NaN,78000.0,NaN,115400,NaN,...,299.476485,NaN,0.331,NaN,NaN,37356.58,NaN,NaN,NaN,NaN
1,"C 202307 80,000( 10)",NaN,콜,NaN,20230713,NaN,80000.0,NaN,115400,NaN,...,299.476485,NaN,0.331,NaN,NaN,35363.25,NaN,NaN,NaN,NaN
2,"C 202307 82,000( 10)",NaN,콜,NaN,20230713,NaN,82000.0,NaN,115400,NaN,...,299.476485,NaN,0.331,NaN,NaN,33370.12,NaN,NaN,NaN,NaN
3,"C 202307 84,000( 10)",NaN,콜,NaN,20230713,NaN,84000.0,NaN,115400,NaN,...,299.476485,NaN,0.331,NaN,NaN,31377.47,NaN,NaN,NaN,NaN
4,"C 202307 86,000( 10)",NaN,콜,NaN,20230713,NaN,86000.0,NaN,115400,NaN,...,299.476485,NaN,0.331,NaN,NaN,29385.78,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,"P 202406 120,000( 10)",NaN,풋,NaN,20240613,NaN,120000.0,NaN,115400,NaN,...,1181.379489,NaN,0.337,NaN,NaN,16034.13,NaN,NaN,NaN,NaN
200,"P 202406 130,000( 10)",NaN,풋,NaN,20240613,NaN,130000.0,NaN,115400,NaN,...,1181.379489,NaN,0.337,NaN,NaN,22140.72,NaN,NaN,NaN,`
201,"P 202406 140,000( 10)",NaN,풋,NaN,20240613,NaN,140000.0,NaN,115400,NaN,...,1181.379489,NaN,0.337,NaN,NaN,29020.68,NaN,NaN,NaN,NaN
202,"P 202406 150,000( 10)",NaN,풋,NaN,20240613,NaN,150000.0,NaN,115400,NaN,...,1181.379489,NaN,0.337,NaN,NaN,36470.17,NaN,NaN,NaN,NaN


In [265]:
# null 값 제거
df= df.dropna(subset=['종목명'])

In [266]:
# 금리 단위 맞추기
df['금리(%)']= df['금리(%)']/100

#### 1. 콜옵션 이론가 구하기

In [267]:
# 콜옵션 이론가 산출 파라미터만 남기기
df = df[df['선물옵션\n구분'] == '콜']

In [268]:
# null값 제거
df = df.dropna(axis = 1)
df

,종목명,선물옵션\n구분,최종\n거래일,행사가격,기초자산\n기준가격,잔존일수,금리(%),배당 현재가치,변동성,이론가격
0,"C 202307 78,000( 10)",콜,20230713,78000.0,115400,32,0.0375,299.476485,0.331,37356.58
1,"C 202307 80,000( 10)",콜,20230713,80000.0,115400,32,0.0375,299.476485,0.331,35363.25
2,"C 202307 82,000( 10)",콜,20230713,82000.0,115400,32,0.0375,299.476485,0.331,33370.12
3,"C 202307 84,000( 10)",콜,20230713,84000.0,115400,32,0.0375,299.476485,0.331,31377.47
4,"C 202307 86,000( 10)",콜,20230713,86000.0,115400,32,0.0375,299.476485,0.331,29385.78
...,...,...,...,...,...,...,...,...,...,...
97,"C 202406 120,000( 10)",콜,20240613,120000.0,115400,368,0.0375,1181.379489,0.331,14439.63
98,"C 202406 130,000( 10)",콜,20240613,130000.0,115400,368,0.0375,1181.379489,0.331,10898.89
99,"C 202406 140,000( 10)",콜,20240613,140000.0,115400,368,0.0375,1181.379489,0.331,8166.59
100,"C 202406 150,000( 10)",콜,20240613,150000.0,115400,368,0.0375,1181.379489,0.331,6023.63


In [269]:
# 이론가격을 구하기 위해 data 자료구조 변경

S = list(map(int,df['기초자산\n기준가격'].tolist()))
K = list(map(int,df['행사가격'].tolist()))
r = list(map (float,df['금리(%)'].tolist()))
div = list(map(int,df['배당  현재가치'].tolist()))
sigma = list(map(float,df['변동성'].tolist()))
t = list(map(int,df['잔존일수'].tolist()))
theo_prc_sys = list(map(float,df['이론가격'].tolist()))
name = df['종목명'].tolist()

In [270]:
# call 옵션 이론가격 구하는 코드

def call(s,k,R,T,Div, Sigma):
  u = np.exp(Sigma*np.sqrt(T/365/49))
  d = np.exp(-Sigma*np.sqrt(T/365/49))
  p = (np.exp(R *T/365/49) - d)/(u-d)
  q = 1-p
  y = []

  for i in range(50):
    C_i = max((s-Div) * u**i * d**(49-i) -k , 0)
    x_i = np.math.factorial(49)/(np.math.factorial(49-i)*np.math.factorial(i))
    p_i = p**i
    q_i = q**(49-i)
    y.append(C_i*x_i* p_i*q_i)

  return sum(y)*np.exp(-R*T/365)

In [271]:
# 모든 행사가격, 모든 월물에 대해 이론가격 구하는 코드
call_price_temp = []

for i in range(len(name)):
  s = S[i]
  k = K[i]
  R = r[i]
  T = t[i]
  Div = div[i]
  Sigma = sigma[i]

  price =  call(s,k,R,T,Div, Sigma)
  call_price_temp.append(price)


In [272]:
# 구한 이론가와 시스템에서 산출한 이론가 비교

call_price = pd.Series(call_price_temp)
df['theo_price_검증용'] = call_price
df['시스템 검증 차이'] = df['이론가격'] - df['theo_price_검증용']

In [273]:
df

,종목명,선물옵션\n구분,최종\n거래일,행사가격,기초자산\n기준가격,잔존일수,금리(%),배당 현재가치,변동성,이론가격,theo_price_검증용,시스템 검증 차이
0,"C 202307 78,000( 10)",콜,20230713,78000.0,115400,32,0.0375,299.476485,0.331,37356.58,37357.057171,-0.477171
1,"C 202307 80,000( 10)",콜,20230713,80000.0,115400,32,0.0375,299.476485,0.331,35363.25,35363.724576,-0.474576
2,"C 202307 82,000( 10)",콜,20230713,82000.0,115400,32,0.0375,299.476485,0.331,33370.12,33370.593447,-0.473447
3,"C 202307 84,000( 10)",콜,20230713,84000.0,115400,32,0.0375,299.476485,0.331,31377.47,31377.941405,-0.471405
4,"C 202307 86,000( 10)",콜,20230713,86000.0,115400,32,0.0375,299.476485,0.331,29385.78,29386.254873,-0.474873
...,...,...,...,...,...,...,...,...,...,...,...,...
97,"C 202406 120,000( 10)",콜,20240613,120000.0,115400,368,0.0375,1181.379489,0.331,14439.63,14439.815764,-0.185764
98,"C 202406 130,000( 10)",콜,20240613,130000.0,115400,368,0.0375,1181.379489,0.331,10898.89,10899.075180,-0.185180
99,"C 202406 140,000( 10)",콜,20240613,140000.0,115400,368,0.0375,1181.379489,0.331,8166.59,8166.734262,-0.144262
100,"C 202406 150,000( 10)",콜,20240613,150000.0,115400,368,0.0375,1181.379489,0.331,6023.63,6023.737167,-0.107167


In [274]:
print('시스템 이론가격과 검증이론가격의 차이가 1원 이상 나는 종목')
df[df['시스템 검증 차이'].abs() > 1]

시스템 이론가격과 검증이론가격의 차이가 1원 이상 나는 종목


,종목명,선물옵션\n구분,최종\n거래일,행사가격,기초자산\n기준가격,잔존일수,금리(%),배당 현재가치,변동성,이론가격,theo_price_검증용,시스템 검증 차이


#### 2. 풋옵션 이론가 구하기

In [247]:
df2 = pd.read_csv(io.BytesIO(uploaded['이론가파람_SK하이닉스.csv']), thousands = ',')


In [248]:
df2 = pd.read_csv(io.BytesIO(uploaded['이론가파람_SK하이닉스.csv']), thousands = ',')
df2 = df2[df2['선물옵션\n구분'] == '풋']
df2['금리(%)'] =  df['금리(%)']/100
df2 = df2.dropna(axis = 1)

In [249]:
df2.reset_index(inplace = True)

In [250]:
df2

,index,종목명,선물옵션\n구분,최종\n거래일,행사가격,기초자산\n기준가격,잔존일수,금리(%),배당 현재가치,변동성,이론가격
0,102,"P 202307 78,000( 10)",풋,20230713,78000.0,115400,32,0.0375,299.476485,0.337,0.05
1,103,"P 202307 80,000( 10)",풋,20230713,80000.0,115400,32,0.0375,299.476485,0.337,0.19
2,104,"P 202307 82,000( 10)",풋,20230713,82000.0,115400,32,0.0375,299.476485,0.337,0.58
3,105,"P 202307 84,000( 10)",풋,20230713,84000.0,115400,32,0.0375,299.476485,0.337,1.61
4,106,"P 202307 86,000( 10)",풋,20230713,86000.0,115400,32,0.0375,299.476485,0.337,3.94
...,...,...,...,...,...,...,...,...,...,...,...
97,199,"P 202406 120,000( 10)",풋,20240613,120000.0,115400,368,0.0375,1181.379489,0.337,16034.13
98,200,"P 202406 130,000( 10)",풋,20240613,130000.0,115400,368,0.0375,1181.379489,0.337,22140.72
99,201,"P 202406 140,000( 10)",풋,20240613,140000.0,115400,368,0.0375,1181.379489,0.337,29020.68
100,202,"P 202406 150,000( 10)",풋,20240613,150000.0,115400,368,0.0375,1181.379489,0.337,36470.17


In [251]:
# 이론가격을 구하기 위해 data 자료구조 변경

S = list(map(int,df2['기초자산\n기준가격'].tolist()))
K = list(map(int,df2['행사가격'].tolist()))
r = list(map (float,df2['금리(%)'].tolist()))
div = list(map(int,df2['배당  현재가치'].tolist()))
sigma = list(map(float,df2['변동성'].tolist()))
t = list(map(int,df2['잔존일수'].tolist()))
theo_prc_sys = list(map(float,df2['이론가격'].tolist()))
name = df2['종목명'].tolist()

In [252]:
# put 옵션 이론가격 구하는 코드

def put(s,k,R,T,Div, Sigma):
  u = np.exp(Sigma*np.sqrt(T/365/49))
  d = np.exp(-Sigma*np.sqrt(T/365/49))
  p = (np.exp(R *T/365/49) - d)/(u-d)
  q = 1-p
  y = []

  for i in range(50):
    P_i = max(k -(s-Div)* u**i * d**(49-i)  , 0)
    x_i = np.math.factorial(49)/(np.math.factorial(49-i)*np.math.factorial(i))
    p_i = p**i
    q_i = q**(49-i)
    y.append(P_i*x_i* p_i*q_i)

  return sum(y)*np.exp(-R*T/365)

In [253]:
# 모든 행사가격, 모든 월물에 대해 이론가격 구하는 코드

put_price_temp = []

for i in range(len(name)):
  s = S[i]
  k = K[i]
  R = r[i]
  T = t[i]
  Div = div[i]
  Sigma = sigma[i]

  price =  put(s,k,R,T,Div,Sigma)
  put_price_temp.append(price)


In [254]:
# 구한 이론가와 시스템에서 산출한 이론가 비교

put_price = pd.Series(put_price_temp)
df2['theo_price_검증용'] = put_price
df2['시스템 검증 차이'] = df2['이론가격'] - df2['theo_price_검증용']

In [255]:
df2

,index,종목명,선물옵션\n구분,최종\n거래일,행사가격,기초자산\n기준가격,잔존일수,금리(%),배당 현재가치,변동성,이론가격,theo_price_검증용,시스템 검증 차이
0,102,"P 202307 78,000( 10)",풋,20230713,78000.0,115400,32,0.0375,299.476485,0.337,0.05,0.051866,-0.001866
1,103,"P 202307 80,000( 10)",풋,20230713,80000.0,115400,32,0.0375,299.476485,0.337,0.19,0.184993,0.005007
2,104,"P 202307 82,000( 10)",풋,20230713,82000.0,115400,32,0.0375,299.476485,0.337,0.58,0.580032,-0.000032
3,105,"P 202307 84,000( 10)",풋,20230713,84000.0,115400,32,0.0375,299.476485,0.337,1.61,1.606160,0.003840
4,106,"P 202307 86,000( 10)",풋,20230713,86000.0,115400,32,0.0375,299.476485,0.337,3.94,3.937248,0.002752
...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,199,"P 202406 120,000( 10)",풋,20240613,120000.0,115400,368,0.0375,1181.379489,0.337,16034.13,16033.936120,0.193880
98,200,"P 202406 130,000( 10)",풋,20240613,130000.0,115400,368,0.0375,1181.379489,0.337,22140.72,22140.525657,0.194343
99,201,"P 202406 140,000( 10)",풋,20240613,140000.0,115400,368,0.0375,1181.379489,0.337,29020.68,29020.449756,0.230244
100,202,"P 202406 150,000( 10)",풋,20240613,150000.0,115400,368,0.0375,1181.379489,0.337,36470.17,36469.899499,0.270501


In [260]:
print('시스템 이론가격과 검증이론가격의 차이가 1원 이상 나는 종목')
df2[df2['시스템 검증 차이'].abs() > 1]

시스템 이론가격과 검증이론가격의 차이가 1원 이상 나는 종목


,index,종목명,선물옵션\n구분,최종\n거래일,행사가격,기초자산\n기준가격,잔존일수,금리(%),배당 현재가치,변동성,이론가격,theo_price_검증용,시스템 검증 차이
